In [43]:
from openai import OpenAI
from bs4 import BeautifulSoup
import requests
from dotenv import load_dotenv
import json
import os
from IPython.display import Markdown, display, update_display

In [44]:
load_dotenv() # we're saving openai api key as env. variable here 
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API Key found !")
elif not api_key.startswith("sk-proj-"):
    print("Not a OpenAI API key")
elif api_key.strip() != api_key:
    print("Remove blank spaces From API key")
else:
    print('API key is found and loaded')

API key is found and loaded


In [45]:
openai = OpenAI()

# **GPT Api Call : To Get IMP Links**

In [46]:
def getlinks(link):
    data = requests.get(link)
    soup = BeautifulSoup(data.content,'html.parser')
    link = link.rstrip('/')
    hrefs = [a['href'] for a in soup.find_all('a', href=True)]
    fix_links = [link+val for val in hrefs if not val.startswith('https')]
    return fix_links

In [47]:
def message_for_ai(link):

    #System Prompt
    system_prompt = "You are provided with links of a website. Your task is to get relevant websites which can be used to create a brochure for a compony"
    system_prompt += "You are to provide response in JSON format"
    system_prompt += """
    Take this as example:
    {
        "links": [
            {"type": "about page", "url": "https://full.url/goes/here/about"},
            {"type": "careers page": "url": "https://another.full.url/careers"}
        ]
    }
    """

    #User Prompt
    user_prompt = str(getlinks(link))
    user_prompt += " These are the links Now find relevant links from this which can be used to make brochure for a compony"

    message = [
    {'role':'system','content':system_prompt},
    {'role':'user','content':user_prompt}]
    
    return message

In [48]:
message_for_ai('https://www.anthropic.com/')

[{'role': 'system',
  'content': 'You are provided with links of a website. Your task is to get relevant websites which can be used to create a brochure for a componyYou are to provide response in JSON format\n    Take this as example:\n    {\n        "links": [\n            {"type": "about page", "url": "https://full.url/goes/here/about"},\n            {"type": "careers page": "url": "https://another.full.url/careers"}\n        ]\n    }\n    '},
 {'role': 'user',
  'content': "['https://www.anthropic.com/', 'https://www.anthropic.com/claude', 'https://www.anthropic.com/claude', 'https://www.anthropic.com/team', 'https://www.anthropic.com/enterprise', 'https://www.anthropic.com/api', 'https://www.anthropic.com/pricing', 'https://www.anthropic.com/research', 'https://www.anthropic.com/company', 'https://www.anthropic.com/careers', 'https://www.anthropic.com/news', 'https://www.anthropic.com/api', 'https://www.anthropic.com/news/3-5-models-and-computer-use', 'https://www.anthropic.com/cl

In [49]:
def get_imp_links(link):
    response = openai.chat.completions.create(
    
        model='gpt-4o-mini',

        messages=message_for_ai(link),
        
        response_format={"type": "json_object"}
    
    )

    result = response.choices[0].message.content

    return json.loads(result)

In [65]:
imp_links = get_imp_links('https://www.anthropic.com/')

In [66]:
imp_links

[{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
 {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
 {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
 {'type': 'enterprise solutions',
  'url': 'https://www.anthropic.com/enterprise'},
 {'type': 'API information', 'url': 'https://www.anthropic.com/api'}]

# **GPT Api Call - To Create a Brochure**

In [78]:
extracted_data = []

for i in imp_links['links']:
    data = requests.get(i.get('url')).content
    soup = BeautifulSoup(data,'html.parser')
    extracted_data.append({"title":i.get('type'),'Content':soup.get_text()})

In [98]:
extracted_data[5] # showing one of output just for example 

{'title': 'pricing page',
 'Content': 'Pricing \\ AnthropicClaudeOverviewTeamEnterpriseAPIPricingResearchCompanyCareersNewsTry ClaudePricing ClaudeAPI TeamPricingResearchCompanyCustomersNewsCareersPress InquiriesSupportStatusAvailabilityTwitterLinkedInYouTubeTerms of Service – ConsumerTerms of Service – CommercialPrivacy PolicyUsage PolicyResponsible Disclosure PolicyCompliancePrivacy Choices© 2024 Anthropic PBC'}

In [93]:
def message_for_ai2():

    system_prompt = "You are given a content of different pages of websites and you need to analyze those content and make a brochure from it in markdown format"
    user_prompt = str(extracted_data) + " Make a brosure for the website from above given data"

    message = [
    {'role':'system','content':system_prompt},
    {'role':'user','content':user_prompt}]
    
    return message

In [ ]:
message_for_ai2() 

In [96]:
def get_brochure():
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=message_for_ai2(),
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [97]:
get_brochure()


# Anthropic: Pioneering Safe AI Solutions

---

## About Us
### Our Mission
At **Anthropic**, we are committed to ensuring that transformative AI technologies positively impact humanity. Our team consists of researchers, engineers, policy experts, and business leaders dedicated to developing reliable, interpretable, and steerable AI systems.

### Our Values
- **Here for the mission**: Our focus is on ensuring AI contributes to societal flourishing.
- **Unusually high trust**: We foster an environment of goodwill, encouraging respect and honesty.
- **One big team**: Collaboration is at the heart of our culture, merging diverse expertise towards shared goals.
- **Pragmatism and empiricism**: We cherish simplicity in problem-solving, aiming for effective solutions.

## Careers at Anthropic
Join a collaborative team that values diverse backgrounds and experiences. We offer:
- **Competitive Compensation**: Fair and competitive salaries, equity packages, and retirement plans.
- **Health & Wellness**: Comprehensive health, dental, vision insurance, and mental health support.
- **Flexible Policies**: Inclusive parental leave and generous paid time off.
- **Career Growth**: Opportunities for professional development and learning.

**Interested? [See open roles](#)**

---

## Products and Services
### Claude: An AI Assistant
**Claude** is designed to enhance workplace productivity, making work smarter rather than harder. Key features include:
- **Easy Collaboration**: Use Claude as a team assistant that taps into collective expertise.
- **Data-Driven Insights**: Summarize complex information using AI to drive innovation.

### Enterprise Solutions
With **Claude for Enterprise**, we power organizations to:
- **Utilize Company Knowledge**: Securely connect Claude to internal data.
- **Boost Productivity**: Enhance efficiency through shared resources and projects.
- **Data Security**: Protect sensitive information with enterprise-grade controls.

---

## Research and Development
At Anthropic, we prioritize **safety research** in AI:
- Explore **novel safety areas** and understand potential impacts of AI.
- Translate research into actionable insights for practical tools and applications.

## Customer Success Stories
Join leading businesses that trust **Claude** for enhanced efficiency and creativity:
- **GitLab**: Accelerates software development.
- **Asana**: Supercharges work management.
- **Deloitte**: Utilizes Claude in trustworthy AI practices.

[Read more success stories](#)

---

## Contact Us
For partnerships, inquiries, or additional information:
- **Email**: support@anthropic.com
- **Follow us on [Twitter](#), [LinkedIn](#), [YouTube](#)**

---

### Legal Information
We value your privacy. For our policies regarding data protection, usage, and processing, please refer to our **[Privacy Policy](#)**.
 

This  brochure captures an overview of Anthropic, highlighting its mission, career opportunities, product offerings, research focus, and customer success stories, along with contact information and legal disclosure.